In [4]:
import numpy as np
import random
import shutil
import cv2
import os
import re

# サイズの取得
def minsize(s):
    
    # 平均画像枚数分ループ（一回しか動かない）
    for i in os.listdir(avePath):
        # 画像の読み込み
        im = cv2.imread(avePath + i)
        # 画像の縦のサイズを取得
        s = int(im.shape[0])
        # ループ終了
        break
    
    # 値を返す
    return s

# ランダムに100枚選出
def ran():
    
    # カウント設定
    count = 0
    
    # 無限ループ
    while True:
        # inspection フォルダ内の枚数を取得
        num = sum(os.path.isfile(os.path.join(tesPath, name)) for name in os.listdir(tesPath))
        
        # もし、フォルダ内の枚数が100枚未満なら
        if num < 100:
            # ランダムに値を取得
            ran  = random.randint(0,3299)
            
            # 選別画像枚数分ループ
            for j in os.listdir(memPath):
                
                # もし、ランダム変数とカウントが一緒ならば
                if count == ran:
                    
                    # Memver_images フォルダ内の all フォルダ内から inspection フォルダにコピー
                    shutil.copy(memPath + j, tesPath)
                    # カウント初期化
                    count = 0
                    # ループ終了
                    break
                    
                # カウントアップ 
                count += 1
        # フォルダ内の枚数が100枚なら
        else:
            # ループの終了
            break
            
# ──　パス　──
newPath = "./Resemble_jedge_record/"
memPath = "./Memver_images/all/"
tesPath = "./inspection/"
avePath = "./averaging/"
ansPath = "./Ans/"

for l in range(1000):
    
    # 結果出力テキスト用パス生成
    wirPath = newPath + str(l + 1)+ "th/"
    # 結果出力テキスト用フォルダ生成
    os.makedirs(wirPath, exist_ok=True)
    
    # 画像100枚選出
    ran()

    # ──　ファイル数取得　──
    #  平均画像の枚数取得（ averaging フォルダ内のファイル数）
    anum = sum(os.path.isfile(os.path.join(avePath, name)) for name in os.listdir(avePath))
    #  分類画像の枚数取得（ inspection フォルダ内のファイル数）
    tnum = sum(os.path.isfile(os.path.join(tesPath, name)) for name in os.listdir(tesPath))

    # 画像サイズ取得
    size = minsize(9999)

    # 総画素数計算
    pnum = size * size

    # ──　配列　──
    
    # 平均画像と選別画像の1ピクセル毎の差 用配列
    # 縦 * 横 * 高さ（3次元配列）
    # 選別画像枚数 * 総画素数 * 平均画像枚数
    ans = np.empty((anum,tnum,pnum))
    # 平均画像 用配列
    # 平均画像枚数 * 総画素数（2次元配列）
    aarray = np.empty((anum,pnum))
    # 選別画像 用配列
    # 選別画像枚数 * 総画素数（2次元配列）
    tarray = np.empty((tnum,pnum))
    # 選別後の誰に誰が何枚選別されかの結果 用配列
    # 平均画像枚数 * 平均画像枚数（1次元配列）
    result = np.zeros(anum * anum)
    # 平均画像と選別画像の差の合計 用配列
    # 平均画像枚数 * 選別画像枚数（2次元配列）
    final = np.zeros((tnum,anum))
    # 差の合計の最小値 用配列
    # 選別画像枚数（1次元配列）
    mini = np.zeros(tnum)
    # 差の最小値の座標 用配列
    index = np.zeros(2)

    # ──　変数宣言　──
    diff = 0
    absdiff = 0
    correctAns = 0
    
    # 配列に値を入れるために必要なカウント用変数
    count = 0
    count2 = 0
    count3 = 0
    horizontal = 0
    

    # 選別ファイルの初期化
    for i in os.listdir(ansPath):
        
        # 選別ファイルの削除
        shutil.rmtree(ansPath + i)
        # 同名で選別ファイルの生成
        os.mkdir(ansPath + i)
        
    # 平均画像の格納（平均画像枚数回分ループ）
    for i in os.listdir(avePath):
        # 平均画像の読み込み
        img = cv2.imread(avePath + i,cv2.IMREAD_GRAYSCALE)
        # ２次元配列を1次元配列に変換し aarray に格納
        aarray[count, :]  =  img.reshape(-1)
        # カウントアップ
        count += 1
    count = 0

    # 選別画像の格納（選別画像枚数回分ループ）
    for i in os.listdir(tesPath):
        # 選別画像をグレースケール化し、読み込み
        img = cv2.imread(tesPath + i,cv2.IMREAD_GRAYSCALE)
        # ２次元配列を1次元配列に変換し tarray に格納
        tarray[count, :] = img.reshape(-1)
        # カウントアップ
        count += 1

    # 平均画像と選別画像の差の計算
    for i in range(pnum):
        for j in range(anum):
            for k in range(tnum):
                
                # 平均画像 n 枚目の n 番目のピクセルと選別画像 n 枚目の n 番目のピクセルを減算
                diff = aarray[j,i] - tarray[k,i]
                # 3次元配列に絶対値に変換した値を格納
                ans[j,k,i] = np.abs(diff)

    # 差の総合の計算
    for j in range(anum):
        for k in range(tnum):
            for i in range(pnum):
                
                # 二次元配列に差の合計を格納
                final[k,j] += ans[j,k,i]

    # 最小値の取得
    for k in range(tnum):
        
        # 結果の最初の値を mini 配列に格納
        mini[k] = final[k,0]
        
        # 平均画像枚数分ループ
        for j in range(anum):
            
            # もし、 n の値が最初の値より小さかったら
            if mini[k] > final[k,j]:
                # 最小値を更新
                mini[k] = final[k,j]

    # 最小値から画像を特定
    for i in os.listdir(tesPath):

        # final 配列の中のどこに mini の値があるかを探して dex 配列に格納
        dex = np.argwhere(final == mini[count2])
        # dex が二次元配列なので指定しやすいよう1次元配列に格納しなおす
        index = np.ravel(dex)
        # カウントアップ
        count2 += 1

        # ファイルの移動
        for j in os.listdir(ansPath):
            # カウントアップ
            count += 1
            
            # もし、カウントと index の横軸と同じなら
            if count == index[1] + 1:
                # inspection フォルダから Ans フォルダに移動
                shutil.move(tesPath + i,ansPath + j)
                # カウント初期化
                count = 0
                # ループ終了
                break
                
    # 結果を書き込み
    for k in os.listdir(ansPath):
        
        # 新しいパスの生成
        txtPath = ansPath + k
        # 新しいテキストフォルダを生成
        t = open(wirPath + k +".txt","w")
        
        # Ansフォルダ内の
        for i in os.listdir(ansPath):
            # メンバー名フォルダ内のファイル名を取得（ j に代入）
            for j in os.listdir(txtPath):
                # 選別された画像の数字を消して配列に代入
                sent = re.split("\d+",j)
                
                # 配列の中身は ["name",".jpg"] となっているので "name" を選択
                if i == sent[0]:
                    # 画像名とフォルダ名が一致した数を数える
                    count3 += 1
            
            # もし、カウントが 0 ではないなら（分類枚数が 0 ではないなら）
            if count3 != 0:
                # テキストに名前と枚数を記入
                t.write(i + " " + str(count3) + "\n")
            
            # result 配列にカウント結果を代入
            result[horizontal] = count3
            
            # count3 の初期化
            count3 = 0
            # 配列 result 特定のための horizontal をプラス１
            horizontal += 1
        # テキストを閉じる
        t.close()
    
    # result 配列を二次元配列に変換
    array = np.reshape(result,(anum,anum))
    
    # 確率の算出
    for i in range(anum):
        correctAns += array[i,i]
    # ファイル名が確率のテキストを生成
    t = open(wirPath + str(correctAns) +"%.txt","w")
    # 閉じる
    t.close()
    
    # 終了合図
    print(str(l + 1) + " ok")

KeyboardInterrupt: 